## LSTM GOLD PREDICT

In [1]:
import pandas as pd

In [11]:
df = pd.read_csv("data/M1_Y1.csv")
df.head()

,time,Open,High,Low,Close,TickVolume,Spread,real_volume
0,2025-07-03 18:02:00+00:00,3328.741,3329.104,3328.741,3329.049,46,160,0
1,2025-07-03 18:03:00+00:00,3329.049,3329.049,3328.882,3329.027,23,160,0
2,2025-07-03 18:04:00+00:00,3329.026,3329.026,3328.598,3328.737,57,160,0
3,2025-07-03 18:05:00+00:00,3328.740,3329.134,3328.647,3329.016,52,160,0
4,2025-07-03 18:06:00+00:00,3329.015,3329.129,3328.962,3328.962,26,160,0


# data pre-processing

In [32]:
def split_by_calendar_bins(df,  minutes=120,time_col='time'):
    
    d = df.copy()
    d[time_col] = pd.to_datetime(d[time_col], errors='coerce')
    d = d.sort_values(time_col)
    groups = d.groupby(pd.Grouper(key=time_col, freq=f'{minutes}min'))
    return [g for _, g in groups if not g.empty]

minute_size = 121
list_of_df = split_by_calendar_bins(df, minute_size)
list_of_df = [g for g in list_of_df if len(g) >= minute_size]



## LSTM Model

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim

In [38]:
class LSTMModel(nn.Module):
    
    def __init__(self, size, embedding_dim, hidden_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(size, embedding_dim)
        
        self.lstm = nn.LSTM(input_size = embedding_dim,
                            hidden_size = hidden_size,
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_size, size)
        
    def init_hidden(self, batch_size, device):
        return (torch.zeroes(1, batch_size, self.hidden_size).to(device),
                torch.zeroes(1, batch_size, self.hidden_size).to(device))
        
    def forward(self, x, state=None, return_state=False):
        x = self.embedding(x)
        
        if state is None:
            state = self.init_hidden(x.size(0), x.device)
        out, state = self.lstm(x, state)
        
        out = self.fc(out)
        return out if not return_state else (out, state)
    

In [ ]:
size = minute_size #idk look up later
embedding_dim = minute_size
hidden_size = 1024
batch_size = 8

if not torch.cuda.is_available():
    raise "cuda not available"

device = torch.device("cuda")

model = LSTMModel(size, embedding_dim, hidden_size).to(device)

print(model)


LSTMModel(
  (embedding): Embedding(121, 121)
  (lstm): LSTM(121, 1024, batch_first=True)
  (fc): Linear(in_features=1024, out_features=121, bias=True)
)
